# Pengenalan Data Pipeline menggunakan TensorFlow

Import library yang akan digunakan seperti berikut

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Kita bisa membuat data secara langsung di dalam memory seperti berikut:

In [ ]:
# Membuat data sederhana
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 6, 8, 10])  # y = 2x

# Membuat dataset dari numpy array
dataset = tf.data.Dataset.from_tensor_slices((x, y))

# Melihat isi dataset
for item in dataset:
    print(f"x: {item[0].numpy()}, y: {item[1].numpy()}")

Beberapa operasi dasar pun bisa kita lakukan secara langsung

In [ ]:
# Membuat dataset yang lebih besar
x_larger = np.arange(20)
y_larger = x_larger * 2

dataset = tf.data.Dataset.from_tensor_slices((x_larger, y_larger))

# 1. Shuffling (mengacak data)
shuffled_dataset = dataset.shuffle(buffer_size=100)

# 2. Batching (mengelompokkan data)
batched_dataset = shuffled_dataset.batch(batch_size=4)

# 3. Repeat dataset
repeated_dataset = batched_dataset.repeat(count=2)

print("Dataset dalam batch:")
for batch in batched_dataset.take(3):
    print(f"Batch x: {batch[0].numpy()}")
    print(f"Batch y: {batch[1].numpy()}\n")

Kita juga bisa meng-mapping suatu fungsi agar di-apply ke seluruh datapoint

In [ ]:
# Fungsi untuk normalisasi data
def normalize(x, y):
    return tf.cast(x, tf.float32) / 20.0, y

# Menerapkan normalisasi ke dataset
normalized_dataset = dataset.map(normalize)

print("Data setelah normalisasi:")
for x, y in normalized_dataset.take(5):
    print(f"x: {x.numpy():.3f}, y: {y.numpy()}")

Data juga bisa dibuat dalam bentuk CSV, namun method ini merupakan bagian dari experimental TensorFlow.

In [ ]:
# Pertama, buat file CSV sederhana untuk contoh
import csv

# Membuat data contoh
with open('sample_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['x', 'y'])
    for i in range(10):
        writer.writerow([i, i*2])

# Membuat dataset dari file CSV
csv_dataset = tf.data.experimental.make_csv_dataset(
    'sample_data.csv',
    batch_size=3,
    label_name='y',
    num_epochs=1
)

print("Data dari CSV:")
for features, labels in csv_dataset.take(3):
    print(f"Features: {features['x'].numpy()}")
    print(f"Labels: {labels.numpy()}\n")

Kita juga bisa melakukan beberapa teknik optimasi dengan melakukan data preproses dan training secara bersamaan. Teknik ini dinamakan prefetching

In [ ]:
# Membuat pipeline lengkap dengan prefetch
BATCH_SIZE = 4
SHUFFLE_BUFFER_SIZE = 100

final_dataset = tf.data.Dataset.from_tensor_slices((x_larger, y_larger))
final_dataset = final_dataset.shuffle(SHUFFLE_BUFFER_SIZE)
final_dataset = final_dataset.batch(BATCH_SIZE)
final_dataset = final_dataset.map(normalize)
final_dataset = final_dataset.prefetch(tf.data.AUTOTUNE)

print("Pipeline lengkap dengan prefetch:")
for x, y in final_dataset.take(2):
    print(f"Batch x: {x.numpy()}")
    print(f"Batch y: {y.numpy()}\n")

Lalu selanjutnya data dapat dimasukkan ke dalam model untuk training.

#### Catatan Penting:

1. `shuffle()`: Mengacak data untuk training yang lebih baik
   - `buffer_size` menentukan seberapa acak datanya
   
2. `batch()`: Mengelompokkan data menjadi batch
   - Penting untuk training yang efisien
   - Ukuran batch memengaruhi memori dan kecepatan training
   
3. `prefetch()`: Mempersiapkan data selanjutnya saat CPU/GPU sedang memproses
   - Menggunakan `tf.data.AUTOTUNE` untuk optimasi otomatis
   
4. `map()`: Mentransformasi data
   - Bisa digunakan untuk preprocessing, augmentasi, dll
   - Dapat diparalelkan dengan `num_parallel_calls`